In [ ]:
from sqlalchemy import select, create_engine, func, text

from telegram_data_models import Message, Chat, MessageTextContent, Entity

from datetime import datetime

import numpy as np
import pandas as pd
from pathlib import Path

from telegram_quality_control.db import get_conn_string

from dotenv import load_dotenv

load_dotenv(".env")

In [ ]:
test_run = False

if test_run:
    chunk_size = 10_000  # number of entities to process in one chunk
else:
    chunk_size = 10_000

entity_type = "hashtag"
entity_type_name = "MessageEntityType.HASHTAG"

In [ ]:
db_url = get_conn_string()

In [ ]:
def download_chunk(num_entities, db_conn_string):
    engine = create_engine(db_conn_string)

    last_entity_id = 0

    while True:
        with engine.connect() as conn:

            sql = (
                select(
                    Entity.id,
                    Entity.message_id,
                    Entity.type,
                    Entity.offset,
                    Entity.length,
                )
                .where(Entity.type.in_([entity_type_name]))
                .where(Entity.id > last_entity_id)
                .order_by(Entity.id)
                .limit(num_entities)
            )

            entities = pd.read_sql_query(sql, conn, index_col="id")

            sql = select(
                MessageTextContent.message_id,
                func.coalesce(MessageTextContent.text, MessageTextContent.caption).label('content'),
            ).where(MessageTextContent.message_id.in_(entities["message_id"].tolist()))

            content = pd.read_sql_query(sql, conn, index_col="message_id")

            if entities.empty:
                break

            last_entity_id = int(entities.index.max())

            yield entities, content


def extract_utf16_substring(text, offset, length):
    utf16_bytes = text.encode('utf-16-le')
    # Each code unit is 2 bytes
    start_byte = offset * 2
    end_byte = (offset + length) * 2
    substring_bytes = utf16_bytes[start_byte:end_byte]
    return substring_bytes.decode('utf-16-le')


def process_chunk(entities, content):
    merged = entities.merge(content, left_on='message_id', right_index=True)
    # Apply the extraction function
    merged[entity_type] = merged.apply(
        lambda row: extract_utf16_substring(row['content'], row['offset'], row['length']), axis=1
    )
    merged["entity_id"] = merged.index
    merged = merged.reset_index(drop=True)
    merged = merged[['message_id', entity_type, 'entity_id']]
    return merged

In [ ]:
if test_run:
    generator = download_chunk(chunk_size, db_url)
    entities, content_df = next(generator)

    print(content_df["content"])

    display(process_chunk(entities, content_df))

In [ ]:
if test_run:
    bmp_only = "Hello 你好 SUBSTRING"
    non_bmp_only = "𝕳𝖊𝖑𝖑𝖔 𝓗𝓮𝓵𝓵𝓸 🌍🚀 SUBSTRING"
    mixed = "Hello🌍世界𝕳𝖊𝖑𝖑𝖔 SUBSTRING"

    print(extract_utf16_substring(bmp_only, 9, 9))
    print(extract_utf16_substring(non_bmp_only, 27, 9))
    print(extract_utf16_substring(mixed, 20, 9))

In [ ]:
engine = create_engine(db_url)

table_name = f"entity_{entity_type}s"

with engine.connect() as conn:
    conn.execute(
        text(
            f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            message_id BIGINT,
            {entity_type} TEXT,
            entity_id BIGINT
        )
    '''
        )
    )
    conn.commit()

In [ ]:
entity_stats = pd.read_sql_query("SELECT MAX(message_id) AS max FROM entities", db_url)

max_message_id = int(entity_stats["max"].iloc[0])

print(f"Max message id: {max_message_id}")

num_rows = 0
average_download_time = []
average_process_time = []

generator = download_chunk(chunk_size, db_url)

tic = datetime.now()

for i, (entities, content) in enumerate(generator):
    tac = datetime.now()

    result = process_chunk(entities, content)
    result.to_sql(table_name, db_url, if_exists='append', index=False)

    toc = datetime.now()

    download_time = (tac - tic).total_seconds()
    process_time = (toc - tac).total_seconds()

    average_download_time.append(download_time)
    average_process_time.append(process_time)

    if test_run and i > 1000:
        break
    if i % 10 == 0:
        current_id = result["message_id"].max()
        frac_ids = current_id / max_message_id * 100
        average_download_time = sum(average_download_time) / len(average_download_time)
        average_process_time = sum(average_process_time) / len(average_process_time)
        print(
            f"[{datetime.now().strftime('%H:%M:%S')}] chunk {i}, \t{frac_ids:.4f}% of ids, \t{average_download_time:.2f} s. download, \t{average_process_time:.2f} s. process"
        )
        average_download_time = []
        average_process_time = []

    tic = datetime.now()